In [1]:


import sys
sys.path.insert(0,'./pyMOE-dev')
# sys.path.insert(0,'../../../7. Tools/pyMOE-dev')
# sys.path.insert(0,'./pyMOE-dev')

from matplotlib import pyplot as plt 
import numpy as np 

from scipy.constants import micro, nano, milli
from scipy import ndimage

# import pyMOE as moe 

In [2]:

def alternate_transitions_symmetric(x, transitions, start_value=1):
    assert np.all(np.diff(transitions) > 0), "transitions should be monotonic increasing"
    y = np.full_like(x, start_value)

    # x_mod = np.mod(x, )
    for i in range(len(transitions)-1):
        mask = np.logical_and(abs(x) >= transitions[i], abs(x) < transitions[i+1])
        y[mask] = start_value * (-1) ** (i+1)
    mask = abs(x) >= transitions[-1]  # Add this line to handle the last transition
    y[mask] = start_value * (-1) ** len(transitions)
    return y


def minimum_step(vector):

    return np.min(np.diff(vector))


def dammann_grating_periodic(x,transitions, period=1, start_value=1):
    # assert np.all(np.array(transitions)<=0.5), "transitions should be <= 0.5"

    transitions = np.array(transitions)*period

    if len(transitions) >1:
        print("Minimum step: %e"%(minimum_step(transitions)))
    # x = clip_remainder(x, period)
    half_period = period/2
    # x = np.mod(np.abs(x+half_period), period)-half_period
    x = np.mod(x, period)

    y = alternate_transitions_symmetric(x,transitions=transitions, start_value=start_value)
    return y



def dammann_2d(XX,YY,transitions_x=None, period_x=1, transitions_y=None, period_y=1, start_value_x=1, start_value_y=1):
    x = XX[0,:]
    y = YY[:,0]

    if transitions_x is not None:
        grating_x = dammann_grating_periodic(x, transitions_x, period=period_x, start_value=start_value_x)
    else:
        grating_x = np.ones_like(x)
    
    if transitions_y is not None:
        grating_y = dammann_grating_periodic(y, transitions_y, period=period_y, start_value=start_value_y)
    else:  
        grating_y = np.ones_like(y)

    grating = np.outer(grating_y, grating_x)
    return grating




In [3]:
transitions_1x11 = [0.06857, 0.20885, 0.44467, 0.52930, 0.72101, 0.72854, 0.86437]
transitions_1x13 = [0.17765, 0.31352, 0.41244, 0.49846, 0.57633, 0.70857, 0.73041]


# Damman mask klayout directly


In [30]:
import pya 
def create_poly_dicing_corner(length, width):
    poly = pya.DPolygon([ 
    pya.DPoint(0, 0), pya.DPoint(0, length),  pya.DPoint(width, length),
    pya.DPoint(width, width),  pya.DPoint(length, width), pya.DPoint(length, width),
    pya.DPoint(length, 0)
    ])
    return poly

def create_corners_cell(layout, field_width, field_height, corner_length, corner_width, layer="layer127"):
    cell_corner_single = layout.create_cell('corner_instance')

    cell_corner = layout.create_cell('corners')
    layer_corners = layout.layer(layer)
    corner_polygon = create_poly_dicing_corner(corner_length, corner_width)

    cell_corner_single.shapes(layer_corners).insert(corner_polygon)


    instance = pya.DCellInstArray(cell_corner_single.cell_index(),
                                pya.DTrans(pya.DTrans.R0, pya.DPoint(-field_width/2, -field_height/2))
                                )
    cell_corner.insert(instance)

    instance = pya.DCellInstArray(cell_corner_single.cell_index(),
                                pya.DTrans(pya.DTrans.R270, pya.DPoint(-field_width/2, field_height/2))
                                )
    cell_corner.insert(instance)


    instance = pya.DCellInstArray(cell_corner_single.cell_index(),
                                pya.DTrans(pya.DTrans.R180, pya.DPoint(field_width/2, field_height/2))
                                )
    cell_corner.insert(instance)

    instance = pya.DCellInstArray(cell_corner_single.cell_index(),
                                pya.DTrans(pya.DTrans.R90, pya.DPoint(field_width/2, -field_height/2))
                                )
    cell_corner.insert(instance)


    cell_corner.flatten(-1, True)

    return cell_corner



def create_label_cell(layout, text, position=(0,0), rotate=True, mag=1000, layer="layer127"):
    

    cell_label = layout.create_cell('label')
    layer_label = layout.layer(layer)


    gen = pya.TextGenerator.default_generator()
    
    region = gen.text(text, layout.dbu, mag)
    if rotate:
        cell_label.shapes(layer_label).insert(region, pya.DTrans(pya.DTrans.R90,pya.DVector(position[0], position[1])))
    else:
        cell_label.shapes(layer_label).insert(region, pya.DTrans(pya.DVector(position[0], position[1])))
    return cell_label



def adjustOrigin(layoutB,topcell):
          bbox= topcell.bbox()
          trans = pya.Trans.new(-bbox.center())  # center point
        #  trans = db.Trans(db.Point((-bbox.left), (-bbox.bottom))) #Lower Left Corner
          for inst in layoutB.top_cell().each_inst():
              layoutB.top_cell().transform(inst,trans)
          for li in layoutB.layer_indices():
            for shape in layoutB.top_cell().each_shape(li):
              layoutB.top_cell().shapes(li).transform(shape,trans)

          layoutB.update()
          return(layoutB)

In [31]:
transitions_1x11 = [0.06857, 0.20885, 0.44467, 0.52930, 0.72101, 0.72854, 0.86437]
transitions_1x13 = [0.17765, 0.31352, 0.41244, 0.49846, 0.57633, 0.70857, 0.73041]

In [40]:
angles_deg =[0.5, 1, 1.5,2 ]

angles = np.radians(angles_deg)
wavelength = 633e-9
m=1

periods = m*wavelength/(np.sin(angles))
periods = periods/1e-6
periods

array([72.53737753, 36.27006982, 24.18158116, 18.13779738])

# Dammann gratings theoretical 11x13



### loop for mutliple angles

In [93]:
# theoretical look up table transitions
transitions_1x11 = [0.06857, 0.20885, 0.44467, 0.52930, 0.72101, 0.72854, 0.86437]
transitions_1x13 = [0.17765, 0.31352, 0.41244, 0.49846, 0.57633, 0.70857, 0.73041]



aperture = 7000



angles_deg =[0.5, 1, 1.5,2 ]


for i, (diff_angle, period) in enumerate(zip(angles_deg, periods)):
    print("Angle: %f deg, Period: %f um"%(diff_angle, period))


    label = "mask%02d_dammann_%0.1fdeg"%(i+1,diff_angle)
    output_filename = label + ".dxf"
    
    angle = np.radians(diff_angle)
    wavelength = 633e-9
    m=1

    period = m*wavelength/(np.sin(angle))
    period = period/1e-6

    # period = int(period*1000)/1000

    
    period_x = period
    period_y = period
    

    Nx = int(aperture/period_x)
    Ny = int(aperture/period_y)
    print(period, Nx,Ny)

    #initialize
    layout = pya.Layout()
    aux_cell = layout.create_cell('aux_cell')

    layers = np.arange(0,128)

    transitions_x = np.concatenate(([0],np.array(transitions_1x13), [1]))*period_x
    transitions_y = np.concatenate(([0],np.array(transitions_1x11), [1]))*period_y

    for t in [transitions_x, transitions_y]:
        difft = np.diff(t)
        # print("Transition distances", difft)

        cd = np.min(difft)
        print("Critical dimension", cd)

    layer_x = 1
    layer_y = 2

    layer_x = layout.layer("dammannhorizontal")
    layer_y = layout.layer("dammannvertical")
    layer_mask = layout.layer("mask")
    cell_dammmann_y = layout.create_cell('dammann_horizontal')

    for i in range(1,len(transitions_y),2):
        start_y = transitions_y[i-1]
        end_y = transitions_y[i]
        # print(start_y, end_y)

        cell_dammmann_y.shapes(layer_y).insert(pya.DBox(0, start_y, period_x, end_y))

    trans = pya.DTrans(0,0)

    # new_instance_y = pya.DCellInstArray(cell_dammmann_y.cell_index(), trans, pya.Vector(period_x, 0 ), pya.Vector(0, period_y), 1,1)
    new_instance_y = pya.DCellInstArray(cell_dammmann_y.cell_index(), trans)
    aux_cell.insert(new_instance_y)

    cell_dammmann_x = layout.create_cell('dammann_horizontal')

    for i in range(1,len(transitions_x),2):
        start_x = transitions_x[i-1]
        end_x = transitions_x[i]
        # print(start_x, end_x)

        cell_dammmann_x.shapes(layer_x).insert(pya.DBox(start_x, 0, end_x, period_y))

    trans = pya.DTrans(0,0)

    # new_instance_x = pya.DCellInstArray(cell_dammmann_x.cell_index(), trans, pya.Vector(period_x, 0 ), pya.Vector(0, period_y), 1,1)
    new_instance_x = pya.DCellInstArray(cell_dammmann_x.cell_index(), trans)
    aux_cell.insert(new_instance_x)



    r1 = pya.Region(aux_cell.begin_shapes_rec(layer_x))
    r2 = pya.Region(aux_cell.begin_shapes_rec(layer_y))

    rxor = r1 ^ r2

    layout.clear_layer(layer_x)
    layout.clear_layer(layer_y)
    # layout.top_cell().shapes(layer_mask).insert(rxor)

    aux_cell.shapes(layer_mask).insert(rxor)



    # Instantiate to top level
    top = layout.create_cell('top')

    # trans = pya.DTrans(pya.Point(0.0,0.0))
    trans = pya.DTrans(0,0)


    instances = pya.DCellInstArray(aux_cell.cell_index(), 
                                   trans, 
                                   pya.DVector(period_x, 0.0 ), 
                                   pya.DVector(0.0, period_y), 
                                   Nx,Ny)
    top.insert(instances)

    adjustOrigin(layout,layout.top_cell())

    # get top cell
    top = layout.top_cell()

    corner_length = 500#*micro
    corner_width = 100#*micro

    field_width = 10000 #*micro
    field_height = 10000 #*micro



    trans = pya.Trans(pya.Point(0,0))


    layer = "mask"
    cell = create_corners_cell(layout, field_width, field_height, corner_length, corner_width, layer)
    top.insert(pya.DCellInstArray(cell.cell_index(), trans))


    cell = create_label_cell(layout, label, position=(-4000, -4000), rotate=False, mag=400, layer=layer)
    top.insert(pya.DCellInstArray(cell.cell_index(), trans))




    layout.write(output_filename)

Angle: 0.500000 deg, Period: 72.537378 um
72.53737753292249 96 96
Critical dimension 1.5842163253190265
Critical dimension 0.5462064528229007
Angle: 1.000000 deg, Period: 36.270070 um
36.27006981958227 192 192
Critical dimension 0.7921383248596783
Critical dimension 0.2731136257414519
Angle: 1.500000 deg, Period: 24.181581 um
24.181581158931913 289 289
Critical dimension 0.5281257325110715
Critical dimension 0.18208730612675694
Angle: 2.000000 deg, Period: 18.137797 um
18.13779738418514 385 385
Critical dimension 0.39612949487060156
Critical dimension 0.13657761430291337


### loop for mutliple angles

In [ ]:
# theoretical look up table transitions
transitions_1x11 = [0.06857, 0.20885, 0.44467, 0.52930, 0.72101, 0.72854, 0.86437]
transitions_1x13 = [0.17765, 0.31352, 0.41244, 0.49846, 0.57633, 0.70857, 0.73041]



aperture = 7000



angles_deg =[0.5, 1, 1.5,2 ]


for i, (diff_angle, period) in enumerate(zip(angles_deg, periods)):
    print("Angle: %f deg, Period: %f um"%(diff_angle, period))


    label = "mask%02d_optim_%0.1fdeg"%(i+1,diff_angle)
    output_filename = label + ".dxf"
    
    angle = np.radians(diff_angle)
    wavelength = 633e-9
    m=1

    period = m*wavelength/(np.sin(angle))
    period = period/1e-6

    # period = int(period*1000)/1000

    
    period_x = period
    period_y = period
    

    Nx = int(aperture/period_x)
    Ny = int(aperture/period_y)
    print(period, Nx,Ny)

    #initialize
    layout = pya.Layout()
    aux_cell = layout.create_cell('aux_cell')

    layers = np.arange(0,128)

    transitions_x = np.concatenate(([0],np.array(transitions_1x13), [1]))*period_x
    transitions_y = np.concatenate(([0],np.array(transitions_1x11), [1]))*period_y

    for t in [transitions_x, transitions_y]:
        difft = np.diff(t)
        # print("Transition distances", difft)

        cd = np.min(difft)
        print("Critical dimension", cd)

    layer_x = 1
    layer_y = 2

    layer_x = layout.layer("dammannhorizontal")
    layer_y = layout.layer("dammannvertical")
    layer_mask = layout.layer("mask")
    cell_dammmann_y = layout.create_cell('dammann_horizontal')

    for i in range(1,len(transitions_y),2):
        start_y = transitions_y[i-1]
        end_y = transitions_y[i]
        # print(start_y, end_y)

        cell_dammmann_y.shapes(layer_y).insert(pya.DBox(0, start_y, period_x, end_y))

    trans = pya.DTrans(0,0)

    # new_instance_y = pya.DCellInstArray(cell_dammmann_y.cell_index(), trans, pya.Vector(period_x, 0 ), pya.Vector(0, period_y), 1,1)
    new_instance_y = pya.DCellInstArray(cell_dammmann_y.cell_index(), trans)
    aux_cell.insert(new_instance_y)

    cell_dammmann_x = layout.create_cell('dammann_horizontal')

    for i in range(1,len(transitions_x),2):
        start_x = transitions_x[i-1]
        end_x = transitions_x[i]
        # print(start_x, end_x)

        cell_dammmann_x.shapes(layer_x).insert(pya.DBox(start_x, 0, end_x, period_y))

    trans = pya.DTrans(0,0)

    # new_instance_x = pya.DCellInstArray(cell_dammmann_x.cell_index(), trans, pya.Vector(period_x, 0 ), pya.Vector(0, period_y), 1,1)
    new_instance_x = pya.DCellInstArray(cell_dammmann_x.cell_index(), trans)
    aux_cell.insert(new_instance_x)



    r1 = pya.Region(aux_cell.begin_shapes_rec(layer_x))
    r2 = pya.Region(aux_cell.begin_shapes_rec(layer_y))

    rxor = r1 ^ r2

    layout.clear_layer(layer_x)
    layout.clear_layer(layer_y)
    # layout.top_cell().shapes(layer_mask).insert(rxor)

    aux_cell.shapes(layer_mask).insert(rxor)



    # Instantiate to top level
    top = layout.create_cell('top')

    # trans = pya.DTrans(pya.Point(0.0,0.0))
    trans = pya.DTrans(0,0)


    instances = pya.DCellInstArray(aux_cell.cell_index(), 
                                   trans, 
                                   pya.DVector(period_x, 0.0 ), 
                                   pya.DVector(0.0, period_y), 
                                   Nx,Ny)
    top.insert(instances)

    adjustOrigin(layout,layout.top_cell())

    # get top cell
    top = layout.top_cell()

    corner_length = 500#*micro
    corner_width = 100#*micro

    field_width = 10000 #*micro
    field_height = 10000 #*micro



    trans = pya.Trans(pya.Point(0,0))


    layer = "mask"
    cell = create_corners_cell(layout, field_width, field_height, corner_length, corner_width, layer)
    top.insert(pya.DCellInstArray(cell.cell_index(), trans))


    cell = create_label_cell(layout, label, position=(-4000, -4000), rotate=False, mag=400, layer=layer)
    top.insert(pya.DCellInstArray(cell.cell_index(), trans))




    layout.write(output_filename)

Angle: 0.500000 deg, Period: 72.537378 um
72.53737753292249 96 96
Critical dimension 1.5842163253190265
Critical dimension 0.5462064528229007
Angle: 1.000000 deg, Period: 36.270070 um
36.27006981958227 192 192
Critical dimension 0.7921383248596783
Critical dimension 0.2731136257414519
Angle: 1.500000 deg, Period: 24.181581 um
24.181581158931913 289 289
Critical dimension 0.5281257325110715
Critical dimension 0.18208730612675694
Angle: 2.000000 deg, Period: 18.137797 um
18.13779738418514 385 385
Critical dimension 0.39612949487060156
Critical dimension 0.13657761430291337


In [ ]:

# Create matrix of squares for measurement

period_x = 50
period_y = 50

Nx = 10
Ny = 20

output_filename = "output.dxf"

label = "P1_13x11_50um_633nm"


#initialize
layout = pya.Layout()
aux_cell = layout.create_cell('aux_cell')

layers = np.arange(0,128)




transitions_x = np.concatenate(([0],np.array(transitions_1x13), [1]))*period_x
transitions_y = np.concatenate(([0],np.array(transitions_1x11), [1]))*period_y

for t in [transitions_x, transitions_y]:
    difft = np.diff(t)
    print("Transition distances", difft)

    cd = np.min(difft)
    print("Critical dimension", cd)

layer_x = 1
layer_y = 2


layer_x = layout.layer("dammannhorizontal")
layer_y = layout.layer("dammannvertical")
layer_mask = layout.layer("mask")
cell_dammmann_y = layout.create_cell('dammann_horizontal')

for i in range(1,len(transitions_y),2):
    start_y = transitions_y[i-1]
    end_y = transitions_y[i]
    # print(start_y, end_y)

    cell_dammmann_y.shapes(layer_y).insert(pya.DBox(0, start_y, period_x, end_y))

trans = pya.Trans(pya.Point(0,0))

new_instance_y = pya.DCellInstArray(cell_dammmann_y.cell_index(), trans, pya.Vector(50, 0 ), pya.Vector(0, 50), 1,1)
aux_cell.insert(new_instance_y)

cell_dammmann_x = layout.create_cell('dammann_horizontal')

for i in range(1,len(transitions_x),2):
    start_x = transitions_x[i-1]
    end_x = transitions_x[i]
    # print(start_x, end_x)

    cell_dammmann_x.shapes(layer_x).insert(pya.DBox(start_x, 0, end_x, period_y))

trans = pya.Trans(pya.Point(0,0))

new_instance_x = pya.DCellInstArray(cell_dammmann_x.cell_index(), trans, pya.Vector(50, 0 ), pya.Vector(0, 50), 1,1)
aux_cell.insert(new_instance_x)



r1 = pya.Region(aux_cell.begin_shapes_rec(layer_x))
r2 = pya.Region(aux_cell.begin_shapes_rec(layer_y))

# r1 = pya.Region(layout.top_cell().begin_shapes_rec(layer_x))
# r2 = pya.Region(layout.top_cell().begin_shapes_rec(layer_y))

rxor = r1 ^ r2

layout.clear_layer(layer_x)
layout.clear_layer(layer_y)
# layout.top_cell().shapes(layer_mask).insert(rxor)

aux_cell.shapes(layer_mask).insert(rxor)



# Instantiate to top level
top = layout.create_cell('top')

trans = pya.Trans(pya.Point(0,0))


instances = pya.DCellInstArray(aux_cell.cell_index(), trans, pya.Vector(period_x, 0 ), pya.Vector(0, period_y), Nx,Ny)
top.insert(instances)

adjustOrigin(layout,layout.top_cell())

# top.flatten(2, True)
# # Merge all instances
# if 1:
#     region = pya.Region(top.begin_shapes_rec(layer_mask))
#     # merges polygons in region
#     region.merge()
#     top.shapes(layer_mask).insert(region)



# get top cell
top = layout.top_cell()

corner_length = 500#*micro
corner_width = 100#*micro

field_width = 10000 #*micro
field_height = 10000 #*micro



trans = pya.Trans(pya.Point(0,0))


layer = "mask"
cell = create_corners_cell(layout, field_width, field_height, corner_length, corner_width, layer)
top.insert(pya.DCellInstArray(cell.cell_index(), trans))


cell = create_label_cell(layout, label, position=(-4000, -4000), rotate=False, mag=400, layer=layer)
top.insert(pya.DCellInstArray(cell.cell_index(), trans))




layout.write(output_filename)


Transition distances [ 8.8825  6.7935  4.946   4.301   3.8935  6.612   1.092  13.4795]
Critical dimension 1.0919999999999987
Transition distances [ 3.4285  7.014  11.791   4.2315  9.5855  0.3765  6.7915  6.7815]
Critical dimension 0.37650000000000006


# Dammann with XOR

In [38]:



# Create matrix of squares for measurement

period_x = 50
period_y = 50

Nx = 10
Ny = 20

output_filename = "output.dxf"

label = "P1_13x11_50um_633nm"


#initialize
layout = pya.Layout()
aux_cell = layout.create_cell('aux_cell')

layers = np.arange(0,128)




transitions_x = np.concatenate(([0],np.array(transitions_1x13), [1]))*period_x
transitions_y = np.concatenate(([0],np.array(transitions_1x11), [1]))*period_y

for t in [transitions_x, transitions_y]:
    difft = np.diff(t)
    print("Transition distances", difft)

    cd = np.min(difft)
    print("Critical dimension", cd)

layer_x = 1
layer_y = 2


layer_x = layout.layer("dammannhorizontal")
layer_y = layout.layer("dammannvertical")
layer_mask = layout.layer("mask")
cell_dammmann_y = layout.create_cell('dammann_horizontal')

for i in range(1,len(transitions_y),2):
    start_y = transitions_y[i-1]
    end_y = transitions_y[i]
    # print(start_y, end_y)

    cell_dammmann_y.shapes(layer_y).insert(pya.DBox(0, start_y, period_x, end_y))

trans = pya.Trans(pya.Point(0,0))

new_instance_y = pya.DCellInstArray(cell_dammmann_y.cell_index(), trans, pya.Vector(50, 0 ), pya.Vector(0, 50), 1,1)
aux_cell.insert(new_instance_y)

cell_dammmann_x = layout.create_cell('dammann_horizontal')

for i in range(1,len(transitions_x),2):
    start_x = transitions_x[i-1]
    end_x = transitions_x[i]
    # print(start_x, end_x)

    cell_dammmann_x.shapes(layer_x).insert(pya.DBox(start_x, 0, end_x, period_y))

trans = pya.Trans(pya.Point(0,0))

new_instance_x = pya.DCellInstArray(cell_dammmann_x.cell_index(), trans, pya.Vector(50, 0 ), pya.Vector(0, 50), 1,1)
aux_cell.insert(new_instance_x)



r1 = pya.Region(aux_cell.begin_shapes_rec(layer_x))
r2 = pya.Region(aux_cell.begin_shapes_rec(layer_y))

# r1 = pya.Region(layout.top_cell().begin_shapes_rec(layer_x))
# r2 = pya.Region(layout.top_cell().begin_shapes_rec(layer_y))

rxor = r1 ^ r2

layout.clear_layer(layer_x)
layout.clear_layer(layer_y)
# layout.top_cell().shapes(layer_mask).insert(rxor)

aux_cell.shapes(layer_mask).insert(rxor)



# Instantiate to top level
top = layout.create_cell('top')

trans = pya.Trans(pya.Point(0,0))


instances = pya.DCellInstArray(aux_cell.cell_index(), trans, pya.Vector(period_x, 0 ), pya.Vector(0, period_y), Nx,Ny)
top.insert(instances)

adjustOrigin(layout,layout.top_cell())

# top.flatten(2, True)
# # Merge all instances
# if 1:
#     region = pya.Region(top.begin_shapes_rec(layer_mask))
#     # merges polygons in region
#     region.merge()
#     top.shapes(layer_mask).insert(region)



# get top cell
top = layout.top_cell()

corner_length = 500#*micro
corner_width = 100#*micro

field_width = 10000 #*micro
field_height = 10000 #*micro



trans = pya.Trans(pya.Point(0,0))


layer = "mask"
cell = create_corners_cell(layout, field_width, field_height, corner_length, corner_width, layer)
top.insert(pya.DCellInstArray(cell.cell_index(), trans))


cell = create_label_cell(layout, label, position=(-4000, -4000), rotate=False, mag=400, layer=layer)
top.insert(pya.DCellInstArray(cell.cell_index(), trans))




layout.write(output_filename)


Transition distances [ 8.8825  6.7935  4.946   4.301   3.8935  6.612   1.092  13.4795]
Critical dimension 1.0919999999999987
Transition distances [ 3.4285  7.014  11.791   4.2315  9.5855  0.3765  6.7915  6.7815]
Critical dimension 0.37650000000000006


In [ ]:



# Create matrix of squares for measurement

period_x = 250
period_y = 250

Nx = 10
Ny = 20

output_filename = "output.dxf"

label = "P1_13x11_50um_633nm"


#initialize
layout = pya.Layout()
aux_cell = layout.create_cell('aux_cell')

layers = np.arange(0,128)




transitions_x = np.concatenate(([0],np.array(transitions_1x13), [1]))*period_x
transitions_y = np.concatenate(([0],np.array(transitions_1x11), [1]))*period_y

layer_x = 1
layer_y = 2


layer_x = layout.layer("dammannhorizontal")
layer_y = layout.layer("dammannvertical")
layer_mask = layout.layer("mask")
cell_dammmann_y = layout.create_cell('dammann_horizontal')

for i in range(1,len(transitions_y),2):
    start_y = transitions_y[i-1]
    end_y = transitions_y[i]
    print(start_y, end_y)

    cell_dammmann_y.shapes(layer_y).insert(pya.DBox(0, start_y, period_x, end_y))

trans = pya.Trans(pya.Point(0,0))

new_instance_y = pya.DCellInstArray(cell_dammmann_y.cell_index(), trans, pya.Vector(50, 0 ), pya.Vector(0, 50), 1,1)
aux_cell.insert(new_instance_y)

cell_dammmann_x = layout.create_cell('dammann_horizontal')

for i in range(1,len(transitions_x),2):
    start_x = transitions_x[i-1]
    end_x = transitions_x[i]
    print(start_x, end_x)

    cell_dammmann_x.shapes(layer_x).insert(pya.DBox(start_x, 0, end_x, period_y))

trans = pya.Trans(pya.Point(0,0))

new_instance_x = pya.DCellInstArray(cell_dammmann_x.cell_index(), trans, pya.Vector(50, 0 ), pya.Vector(0, 50), 1,1)
aux_cell.insert(new_instance_x)



r1 = pya.Region(aux_cell.begin_shapes_rec(layer_x))
r2 = pya.Region(aux_cell.begin_shapes_rec(layer_y))

# r1 = pya.Region(layout.top_cell().begin_shapes_rec(layer_x))
# r2 = pya.Region(layout.top_cell().begin_shapes_rec(layer_y))

rxor = r1 ^ r2

layout.clear_layer(layer_x)
layout.clear_layer(layer_y)
# layout.top_cell().shapes(layer_mask).insert(rxor)

aux_cell.shapes(layer_mask).insert(rxor)



# Instantiate to top level
top = layout.create_cell('top')

trans = pya.Trans(pya.Point(0,0))


instances = pya.DCellInstArray(aux_cell.cell_index(), trans, pya.Vector(period_x, 0 ), pya.Vector(0, period_y), Nx,Ny)
top.insert(instances)

adjustOrigin(layout,layout.top_cell())

# top.flatten(2, True)
# # Merge all instances
# if 1:
#     region = pya.Region(top.begin_shapes_rec(layer_mask))
#     # merges polygons in region
#     region.merge()
#     top.shapes(layer_mask).insert(region)



# get top cell
top = layout.top_cell()

corner_length = 500#*micro
corner_width = 100#*micro

field_width = 10000 #*micro
field_height = 10000 #*micro



trans = pya.Trans(pya.Point(0,0))


layer = "mask"
cell = create_corners_cell(layout, field_width, field_height, corner_length, corner_width, layer)
top.insert(pya.DCellInstArray(cell.cell_index(), trans))


cell = create_label_cell(layout, label, position=(-4000, -4000), rotate=False, mag=400, layer=layer)
top.insert(pya.DCellInstArray(cell.cell_index(), trans))




layout.write(output_filename)


0.0 17.142500000000002
52.2125 111.1675
132.325 180.2525
182.135 216.0925
0.0 44.4125
78.38000000000001 103.11
124.61500000000001 144.0825
177.1425 182.6025


In [31]:
r2

: 

In [ ]:

corner_length = 500#*micro
corner_width = 100#*micro

field_width = 48000 #*micro
field_height = 25000 #*micro
field_height = 15000 #*micro

layer = "layer127"
# gscalibration.add_corners(field_width, field_height, corner_length, corner_width, layer)
# label = "text"
# # gscalibration.add_label(label, position=(-4000, -4000), mag=400, layer=layer)


# trans = pya.Trans(pya.Point(0,0))

# top_cell = layout.top_cell()

# cell_corner = create_corners_cell(layout, field_width, field_height, corner_length, corner_width, layer)
# top_cell.insert(pya.DCellInstArray(cell_corner.cell_index(), trans))


# for i in range (16):
#     x0,y0 = -22500, 7250
#     position=(x0+step*i, y0)
#     rotate=False
#     mag=200

#     label = "%d"%(i)
#     cell_label = create_label_cell(layout, label, position, rotate, mag, layer)
#     top_cell.insert(pya.DCellInstArray(cell_label.cell_index(), trans))


layout.write("test.dxf")



# layout.write("gs_calibration_v20240507.dxf")

In [ ]:

#initialize
layout = pya.Layout()
top = layout.create_cell('top')



# single_calibration = layout.create_cell('rectangle')
# layer_n = layout.layer("layer_n")


layers = np.arange(0,128)




# Create matrix of squares for measurement

period_x = 50
period_y = 50


transitions_x = np.concatenate(([0],np.array(transitions_1x13), [1]))*period_x
transitions_y = np.concatenate(([0],np.array(transitions_1x11), [1]))*period_y

layer_x = 1
layer_y = 2


layer_x = layout.layer("dammannhorizontal")
layer_y = layout.layer("dammannvertical")
cell_dammmann_y = layout.create_cell('dammann_horizontal')

for i in range(1,len(transitions_y),2):
    start_y = transitions_y[i-1]
    end_y = transitions_y[i]
    print(start_y, end_y)

    cell_dammmann_y.shapes(layer_y).insert(pya.DBox(0, start_y, period_x, end_y))

trans = pya.Trans(pya.Point(0,0))

new_instance = pya.DCellInstArray(cell_dammmann_y.cell_index(), trans, pya.Vector(50, 0 ), pya.Vector(0, 50), 1,1)
top.insert(new_instance)

cell_dammmann_x = layout.create_cell('dammann_horizontal')

for i in range(1,len(transitions_x),2):
    start_x = transitions_x[i-1]
    end_x = transitions_x[i]
    print(start_x, end_x)

    cell_dammmann_x.shapes(layer_x).insert(pya.DBox(start_x, 0, end_x, period_y))

trans = pya.Trans(pya.Point(0,0))

new_instance = pya.DCellInstArray(cell_dammmann_x.cell_index(), trans, pya.Vector(50, 0 ), pya.Vector(0, 50), 1,1)
top.insert(new_instance)




corner_length = 500#*micro
corner_width = 100#*micro

field_width = 48000 #*micro
field_height = 25000 #*micro
field_height = 15000 #*micro

layer = "layer127"
# gscalibration.add_corners(field_width, field_height, corner_length, corner_width, layer)
# label = "text"
# # gscalibration.add_label(label, position=(-4000, -4000), mag=400, layer=layer)


# trans = pya.Trans(pya.Point(0,0))

# top_cell = layout.top_cell()

# cell_corner = create_corners_cell(layout, field_width, field_height, corner_length, corner_width, layer)
# top_cell.insert(pya.DCellInstArray(cell_corner.cell_index(), trans))


# for i in range (16):
#     x0,y0 = -22500, 7250
#     position=(x0+step*i, y0)
#     rotate=False
#     mag=200

#     label = "%d"%(i)
#     cell_label = create_label_cell(layout, label, position, rotate, mag, layer)
#     top_cell.insert(pya.DCellInstArray(cell_label.cell_index(), trans))


layout.write("test.dxf")



# layout.write("gs_calibration_v20240507.dxf")

0.0 3.4285
10.4425 22.2335
26.465 36.0505
36.427 43.2185
0.0 8.8825
15.676000000000002 20.622
24.923000000000002 28.8165
35.4285 36.5205


In [17]:

l1 = layer_x
l2 = layer_y

r1 = pya.Region(layout.top_cell().begin_shapes_rec(l1))
r2 = pya.Region(layout.top_cell().begin_shapes_rec(l2))

rxor = r1 ^ r2

layout.clear_layer(l1)
layout.clear_layer(l2)
layout.top_cell().shapes(l1).insert(rxor)

layout.write("output.gds")